In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
cid = '6b4356b846d64135bf4bcec341a96b9c'
secret = '0baa0aec07994419887012b77df4a93e'

In [3]:
#Authentication - without user  
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [4]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [5]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist / this would only take information of the first artist, maybe a song could have more than one
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"] 

In [42]:
sp.audio_features(track_uri)[0]

{'danceability': 0.315,
 'energy': 0.715,
 'key': 11,
 'loudness': -8.072,
 'mode': 0,
 'speechiness': 0.0362,
 'acousticness': 0.00125,
 'instrumentalness': 7.97e-06,
 'liveness': 0.0942,
 'valence': 0.497,
 'tempo': 155.925,
 'type': 'audio_features',
 'id': '6Qyc6fS4DsZjB2mRW9DsQs',
 'uri': 'spotify:track:6Qyc6fS4DsZjB2mRW9DsQs',
 'track_href': 'https://api.spotify.com/v1/tracks/6Qyc6fS4DsZjB2mRW9DsQs',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6Qyc6fS4DsZjB2mRW9DsQs',
 'duration_ms': 289533,
 'time_signature': 3}

In [12]:
sp.playlist_tracks(playlist_URI)['items'][0]['track'].keys()

dict_keys(['preview_url', 'available_markets', 'explicit', 'type', 'episode', 'track', 'album', 'artists', 'disc_number', 'track_number', 'duration_ms', 'external_ids', 'external_urls', 'href', 'id', 'name', 'popularity', 'uri', 'is_local'])

In [30]:
artist_uri = sp.playlist_tracks(playlist_URI)['items'][0]['track']['artists'][0]['uri']
artist = sp.artist(artist_uri)

In [26]:
sp.playlist_tracks(playlist_URI)['items'][0]['track']['name']

'Die With A Smile'

In [20]:
artist.keys()

dict_keys(['external_urls', 'followers', 'genres', 'href', 'id', 'images', 'name', 'popularity', 'type', 'uri'])

In [31]:
artist['popularity']

91